# Protocol Example

<div class="alert alert-block alert-info">

This example illustrates how to work with the Protocol API in the <code>biopsykit.protocols</code> module. These classes represent (psychological) protocols. One instance of these protocols represents a particular study the protocol was conducted, and data collected during the study.  
    
The Protocol API further offers functions to process different kind of data collected during that study without the need of setting up "common" processing pipelines each time (see the [<code>ECG_Analysis_Example.ipynb</code>](ECG_Analysis_Example.ipynb) for further information on data processing pipelines).  
    
As input, this notebook uses the heart rate processing outputs generated from the ECG processing pipeline as shown in [<code>ECG_Processing_Example.ipynb</code>](ECG_Processing_Example.ipynb)) as well as saliva data.
    
</div>

## Setup and Helper Functions

In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.protocols import BaseProtocol, MIST, TSST

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

In [ ]:
def display_dict_structure(dict_to_display):
    _display_dict_recursive(dict_to_display)
    
def _display_dict_recursive(dict_to_display):
    if isinstance(dict_to_display, dict):
        display(dict_to_display.keys())
        _display_dict_recursive(list(dict_to_display.values())[0])
    else:
        display("Dataframe shape: {}".format(dict_to_display.shape))
        display(dict_to_display.head())

## Overview

The Protocol API of `BioPsyKit` offers classes that represent various psychological protocols. 

The classes allow to add *time-series data* (such as heart rate) as well as *tabular data* (such as saliva) to an instance of a `Protocol` object. Further, it offers functions to process the data and aggregate the results on a study level (such as computing mean and standard error over all subjects within different phases during the protocol).  

For this, it uses functions from the `biopsykit.utils.data_processing` module to split data, compute aggregated results, etc. While these functions can also be used stand-alone, without using the Protocol API, it is, however, recommended to make use of the Protocol API whenever possible since you don't have to take care of calling each function manually, and in the right order. 

See the `ECG_Analysis_Example.ipynb` notebook for further examples.

## General Usage – `BaseProtocol`

The base class for all protocols is `BaseProtocol`. It provides the general functionality. Besides that, there exist a couple of predefined subclasses that represent standardized psychological protocols, such as `MIST` (Montreal Imaging Stress Task) and `TSST` (Trier Social Stress Test).

Each protocol is expected to have a certain structure which can be specified by passing a dictionary describing the protocol structure when creating the Protocol object. The structure can be nested, up to three nested structure levels are supported:

* 1st level: ``study part``: These can be different, disjunct parts of the complete study, such as: "Preface", "Test", and "Questionnaires"
* 2nd level: ``phase``: One *study part* of the psychological protocol can consist of different *phases*, such as: "Preparation", "Stress", "Recovery"
* 3rd level: ``subphase``: One *phase* can consist of different *subphases*, such as: "Baseline", "Arithmetic Task", "Feedback"

The following examples illustrate different possibilities of defining the protocol structure:

In [ ]:
# Example 1: study with three parts, no finer division into phases
structure = {
    "Preface": None,
    "Test": None,
    "Questionnaires": None
}
BaseProtocol(name="Base", structure=structure)

In [ ]:
# Example 2: study with three parts, all parts have different phases with specific durations
structure = {
    "Preface": {"Questionnaires": 240, "Baseline": 60},
    "Test": {"Preparation": 120, "Test": 240, "Recovery": 120},
    "Recovery": {"Part1": 240, "Part2": 240}
}
BaseProtocol(name="Base", structure=structure)

In [ ]:
# Example 3: only certain study parts have different phases (example: TSST)
structure = {
    "Before": None,
    "TSST": {"Preparation": 300, "Talk": 300, "Math": 300},
    "After": None
}
BaseProtocol(name="Base", structure=structure)

In [ ]:
# Example 4: study with phases and subphases, only certain study parts have different phases (example: MIST)
structure = {
    "Before": None,
    "MIST": {
        "MIST1": {"BL": 60, "AT": 240, "FB": 120},
        "MIST2": {"BL": 60, "AT": 240, "FB": 120},
        "MIST3": {"BL": 60, "AT": 240, "FB": 120}
    },
    "After": None
}
BaseProtocol(name="Base", structure=structure)

### Load Data

#### Heart Rate and R Peak Data

Load processed heart rate time-series data of all subjects and concatenate into one dict.

In [ ]:
subject_data_dict_hr = {}
# or use your own path
ecg_path = bp.example_data.get_ecg_processing_results_path_example()

for file in sorted(ecg_path.glob("hr_result_*.xlsx")):
    subject_id = re.findall("hr_result_(Vp\w+).xlsx", file.name)[0]
    subject_data_dict_hr[subject_id] = pd.read_excel(file, sheet_name=None, index_col="time")
    
subject_data_dict_rpeaks = {}
for file in sorted(ecg_path.glob("rpeaks_result_*.xlsx")):
    subject_id = re.findall("rpeaks_result_(Vp\w+).xlsx", file.name)[0]
    subject_data_dict_rpeaks[subject_id] = pd.read_excel(file, sheet_name=None, index_col="time")

Structure of the `SubjectDataDict`:

In [ ]:
display_dict_structure(subject_data_dict_hr)

#### Subject Conditions
(will be needed later)

**Note**: This is only an example, thus, the condition list is manually constructed. Usually, you should import a condition list from a file.

In [ ]:
condition_list = pd.DataFrame(
    ["Control", "Intervention"], 
    columns=["condition"], 
    index=pd.Index(["Vp01", "Vp02"], name="subject")
)
condition_list

### Create `Protocol` instance

Let's assume our example protocol consists of only one study part, so we can ignore this level. The study consists of four different phases: "Baseline", "Intervention", "Stress", and "Recovery". Each of this phase has, in turn, three subphases: "Start", "Middle", and "End" with the durations 20, 30, and 10 seoncds, respectively. Our structure dict then looks like this:

In [ ]:
subphases =  {"Start": 20, "Middle": 30, "End": 10}
structure = {
    "Baseline": subphases,
    "Intervention": subphases,
    "Stress": subphases,
    "Recovery": subphases
}

In [ ]:
base_protocol = BaseProtocol(name="Base", structure=structure)

### Add Heart Rate Data

The data to be added to the Protocol object is expected to be a `SubjectDataDict`, a special nested dictionary structure that contains processed data of all subjects. The dictionaries `subject_data_dict_hr` and `subject_data_dict_rpeaks` are already in this structure, so they can be added to the Protocol object. 

Since we only have one study part, we can directly add this data without specifying a `study_part` parameter. The data is added to a *study part* named "Study" for better internal handling. The added data can be accessed via the `hr_data` property.

In [ ]:
base_protocol.add_hr_data(hr_data=subject_data_dict_hr, rpeak_data=subject_data_dict_rpeaks)

In [ ]:
display_dict_structure(base_protocol.hr_data)

### Compute *Aggregated Results*

Next, we can compute *Aggregated Results*. For this, we can use the method `BaseProtocol.compute_hr_results()` which allows us to enable/disable the specific processing steps and provide different parameters to the individual step (see the `ECG_Analysis_Example.ipynb` notebook for further explanations on the different processing steps).

Since different kinds of aggregated results can be computed (e.g., aggregating only over phases vs. aggregating over subphases, normalization of heart rate data vs. no normalization, ...) we can speficy an identifier for these results via the `result_id` parameter.

`BaseProtocol.compute_hr_results()` can do the following steps (`True`/`False` indicates which steps are enabled by default – have also a look at the function documentation!):

* Resample (`resample_sec`): `True`
* Normalize (`normalize_to`): `False`
* Select Phases (`select_phases`): `False`
* Split into Subphases (`split_into_subphases`: `False`
* Aggregate per Subject (`mean_per_subject`): `True`
* Add Conditions (`add_conditions`): `False`

#### Aggregate over Phases (Without Normalization)

In [ ]:
base_protocol.compute_hr_results(
    "hr_phases"
)

#### Aggregate over Phases, Add Subject Conditions

In [ ]:
base_protocol.compute_hr_results(
    "hr_phases_cond", 
    add_conditions=True,
    params={"add_conditions": condition_list}
)

#### Aggregate over Phases (With Normalization)

If, for example, we want to normalize the heart rate data to the "Baseline" phase, we can enable Normalization (`normalize_to`). Since all data were normalized to the "Baseline" phase we can drop this phase for later analysis by enabling the `select_phases` step. The parameters for these steps is supplied via the `params` dict:

In [ ]:
base_protocol.compute_hr_results(
    "hr_phases_norm",
    normalize_to=True,
    select_phases=True,
    params={
        "normalize_to": "Baseline",
        "select_phases": ["Intervention", "Stress", "Recovery"]
    }
)

#### Aggregate over Subphases (With Normalization)

In [ ]:
base_protocol.compute_hr_results(
    "hr_subphases_norm",
    normalize_to=True,
    select_phases=True,
    split_into_subphases=True,
    params={
        "normalize_to": "Baseline",
        "select_phases": ["Intervention", "Stress", "Recovery"],
        "split_into_subphases": subphases
    }
)

#### Access Results

The results can then be accessed via the `hr_results` property:

In [ ]:
base_protocol.hr_results["hr_phases"].head()

In [ ]:
base_protocol.hr_results["hr_phases_cond"].head()

In [ ]:
base_protocol.hr_results["hr_phases_norm"].head()

In [ ]:
base_protocol.hr_results["hr_subphases_norm"].head()

### Compute *Ensemble Time-Series*

Next, we can compute *Ensemble Time-Series*. For this, we can use the method `BaseProtocol.compute_hr_ensemble()` which allows us to enable/disable the specific processing steps and provide different parameters to the individual step (see the `ECG_Analysis_Example.ipynb` notebook for further explanations on the different processing steps).

Since different kinds of ensemble time-series can be computed (e.g., normalization of heart rate data vs. no normalization, ...), we can speficy an identifier for these results via the `ensemble_id` parameter.

`BaseProtocol.compute_hr_ensemble()` can do the following steps (`True`/`False` indicates which steps are enabled by default – have also a look at the function documentation!):

* Resample (`resample_sec`): `True`
* Normalize (`normalize_to`): `True`
* Select Phases (`select_phases`): `False`
* Cut Phases to Shortest Duration (`cut_phases`): `True`
* Merge Dictionary (`merge_dict`): `True`
* Add Conditions (`add_conditions`): `False`


In [ ]:
base_protocol.compute_hr_ensemble(
    "hr_ensemble",
    params={
        "normalize_to": "Baseline"
    }
)

In [ ]:
display_dict_structure(base_protocol.hr_ensemble["hr_ensemble"])

### Compute *HRV Parameters*

Next, we can compute *HRV Parameters*. For this, we can use the method `BaseProtocol.compute_hrv_results()` which allows us to enable/disable the specific processing steps and provide different parameters to the individual step (see the `ECG_Analysis_Example.ipynb` notebook for further explanations on the different processing steps).

Since different kinds of HRV results can be computed (e.g., aggregating only over phases vs. aggregating over subphases, ...) we can speficy an identifier for these results via the `result_id` parameter.

`BaseProtocol.compute_hrv_results()` can do the following steps (`True`/`False` indicates which steps are enabled by default – have also a look at the function documentation!):

* Select Phases (`select_phases`): `False`
* Split into Subphases (`split_into_subphases`): `False`
* Add Conditions (`add_conditions`): `False`

Additionally, the function has arguments to further specify HRV processing:
* `dict_levels`: The names of the dictionary levels. Corresponds to the index level names of the resulting, concatenated dataframe. Default: `None` for the levels ["subject", "phase"] (or ["subject", "phase", "subphase"], when `split_into_subphases` is `True`)
* (`hrv_params`): Dictionary with parameters to configure HRV parameter computation. Parameters are passed to `EcgProcessor.hrv_process()`. Examples:
  * `hrv_types`: list of HRV types to be computed. Subset of ["hrv_time", "hrv_nonlinear", "hrv_frequency"].
  * `correct_rpeaks`: Whether to apply R peak correction before computing HRV parameters or not

#### Compute HRV Parameter over Phases

In [ ]:
base_protocol.compute_hrv_results(
    "hrv_phases"
)

In [ ]:
base_protocol.hrv_results["hrv_phases"].head()

#### Compute HRV Parameter over Subphases

Assuming we only want to compute a certain type of HRV measures (in this example, only time-domain measures), we can specify this in the `hrv_params` argument.

In [ ]:
base_protocol.compute_hrv_results(
    "hrv_subphases",
    split_into_subphases=True,
    params={
        "split_into_subphases": subphases
    },
    hrv_params={
        "hrv_types": "hrv_time"
    }
)

In [ ]:
base_protocol.hrv_results["hrv_subphases"].head()

### Add Precomputed Results

If processing results were already computed (e.g., in another notebook or because it's data from an old study), the precomputed results can be imported and directly added to the `Protocol` instance.

#### Ensemble Time-series

In this example, we have *Ensemble Time-series* data from a study that has 3 phases (*Phase1*, *Phase2*, *Phase3*). This data is already in the correct format, so we can directly add it to a `Protocol` object (that also has the correct structure – the duration of the single phases is not of importance, so we just leave it empty here).

In [ ]:
dict_merged_norm = bp.example_data.get_hr_ensemble_sample()

In [ ]:
base_protocol = BaseProtocol(
    "TimeSeriesDemo", 
    structure={
        "Phase1": None,
        "Phase2": None,
        "Phase3": None
    }
)

In [ ]:
base_protocol.add_hr_ensemble("hr_ensemble", dict_merged_norm)

In [ ]:
display_dict_structure(base_protocol.hr_ensemble["hr_ensemble"])

## MIST

The `MIST` represents the *Montreal Imaging Stress Task*. 

Typically, the MIST has the following structure:
* "Before": The study part before conducting the "actual" MIST. Here, subjects typically fill out questionnaires and provide a neutral baseline before stress induction
* "MIST": The "actual" MIST. The MIST protocol consists of three repetitions of the same subphases: *Baseline* (BL), *Arithmetic Tasks* (AT), *Feedback* (FB). Each subphase has a duration that is specified in seconds.
* "After": The study part after the MIST. Here, subjects typically fill out further questionnaires and remain in the examination room to provide further saliva samples.

In summary:
* *Study Parts*: Before, MIST, After
* *Phases*: MIST1, MIST2, MIST3
* *Subphases*: BL, AT, FB
* *Subphase Durations*: 60, 240, 0 (Feedback Interval has length 0 because it's of variable length for each MIST phase and will later be inferred from the data)


Hence, the `structure` dictionary would look like the following:

```
structure = {
    "Before": None,
    "MIST": {
        "MIST1": {"BL": 60, "AT": 240, "FB": 0},
        "MIST2": {"BL": 60, "AT": 240, "FB": 0},
        "MIST3": {"BL": 60, "AT": 240, "FB": 0}
    },
    "After": None
}
```

In [ ]:
structure = {
    "Before": None,
    "MIST": {
        "MIST1": ["BL", "AT", "FB"],
        "MIST2": ["BL", "AT", "FB"],
        "MIST3": ["BL", "AT", "FB"],
    },
    "After": None
}

In [ ]:
mist = MIST(structure=structure)
mist

## TSST

The `TSST` represents the *Trier Social Stress Test*. 

Typically, the TSST has the following structure:
* "Before": The study part before conducting the "actual" TSST. Here, subjects typically fill out questionnaires and provide a neutral baseline before stress induction
* "TSST": The "actual" TSST. The TSST protocol consists two phases: *Talk* and *Math*, both with a duration of 5 minutes (300 seconds).
* "After": The study part after the TSST. Here, subjects typically fill out further questionnaires and remain in the examination room to provide further saliva samples.

In summary:
* *Study Parts*: Before, TSST, After
* *Phases*: Talk, Math
* *Phase Durations*: 300, 300


Hence, the `structure` dictionary would look like the following:

```
structure = {
    "Before": None,
    "TSST": {
        "Talk": 300,
        "Math": 300,
    },
    "After": None
}
```

In [ ]:
structure = {
    "Before": None,
    "TSST": {
        "Talk": 300,
        "Math": 300,
    },
    "After": None
}

In [ ]:
tsst = TSST(structure=structure)
tsst

## Plotting

The Protocol API offers functions to directly visualize the processed study data of the `Protocol` instance. So far, the visualization of heart rate data (`hr_mean_plot()` and `hr_ensemble_plot()`) and saliva data (`saliva_plot()`) is supported.

Create Protocol object

*Note*: For this example we assume that data was collected during the MIST.

In [ ]:
mist = MIST()
mist

### Aggregated Results

Load Example Aggregated Results Data

In [ ]:
hr_result = bp.example_data.get_hr_result_sample()
# Alternatively: Load your own aggregated results
# hr_result = bp.io.load_long_format_csv("<path-to-aggregated-results-in-long-format>")

In [ ]:
hr_result.head()

Add Aggregated Results Data

In [ ]:
mist.add_hr_results("hr_result", hr_result)

Plot Mean HR Plot

In [ ]:
fig, ax = plt.subplots()
mist.hr_mean_plot("hr_result", ax=ax)

### Ensemble Time-series

#### Load Example Ensemble Data

In [ ]:
dict_hr_ensemble = bp.example_data.get_hr_ensemble_sample()

# Alternatively: Load your own ensemble time-series data
# dict_hr_ensemble = bp.io.load_pandas_dict_excel("<path-to-heart-rate-ensemble-data.xlsx>")

# Note: We rename the phase names (Phase1-3) of this example dataset to match 
# it with the standard phase names of the MIST (MIST1-3)
dict_hr_ensemble = {key: value for key, value in zip(["MIST1", "MIST2", "MIST3"], dict_hr_ensemble.values())}

In [ ]:
display_dict_structure(dict_hr_ensemble)

#### Add Ensemble Data

In [ ]:
mist.add_hr_ensemble("hr_ensemble", dict_hr_ensemble)

#### Plot HR Ensemble Plot

*Note*: Since the MIST contains subphases the `MIST.hr_ensemble_plot()` function makes use of `structure` property of the `MIST` object and automatically highlights the subphases in the HR Ensemble Plot.

In [ ]:
fig, ax = plt.subplots()
mist.hr_ensemble_plot("hr_ensemble", ax=ax)

### Saliva

Just like heart rate data we can also add saliva data to the `Protocol` object. We just need to specify which type of data it is (e.g., "cortisol", "amylase", etc.) – see the `Saliva_Example.ipynb` notebook for further information.

Saliva data can be added to the `Protocol` object in two different format:
* Saliva data per subject as `SalivaRawDataFrame`, a specialized dataframe containing "raw" (unaggregated) saliva data in a standardized format. `BioPsyKit` will then interanally compute mean and standard error.
* Aggregated saliva data (mean and standard error) as `SalivaMeanSeDataFrame`, a specialized dataframe containing mean and standard error of saliva samples in a standardized format.

#### SalivaRawDataFrame

Load Example Cortisol Data in `SalivaRawDataFrame` format.

*Note*: The sample times are provided relative to the psychological protocol. The first sample is dropped for plotting.

In [ ]:
saliva_data = bp.example_data.get_saliva_example(sample_times=[-30, -1, 0, 10, 20, 30, 40])
# alternatively: load your own saliva data
# saliva_data = bp.io.saliva.load_saliva_wide_format("<path-to-saliva-data>")
# drop first saliva sample (not needed because it was only assessed to control for high baseline)
saliva_data = saliva_data.drop("0", level="sample")
saliva_data.head()

Add Saliva Data to `MIST` object

In [ ]:
mist.add_saliva_data(saliva_data, "cortisol")

In [ ]:
fig, ax = mist.saliva_plot(saliva_type="cortisol")
fig.tight_layout()

#### SalivaMeanSeDataFrame

Create Protocol object – For this example, we're assuming the data was collected during the TSST.

In [ ]:
structure = {
    "Before": None,
    "TSST": {
        "Talk": 300,
        "Math": 300,
    },
    "After": None
}

tsst = TSST(structure=structure)
tsst

Load Example Saliva Data (cortisol, amylase, and IL-6) in `SalivaMeanSeDataFrame` format (or, more presicely, a dictionary of such).

In [ ]:
saliva_dict = bp.example_data.get_saliva_mean_se_example()
display_dict_structure(saliva_dict)

Add Saliva Data to `TSST` object. Since it's a dictionary, we don't have to specify the `saliva_type` – the names are automatically inferred from the dictionary keys (we can, however, also manually specify the saliva type if we want).

In [ ]:
tsst.add_saliva_data(saliva_dict)
display_dict_structure(tsst.saliva_data)

Plot Cortisol Data

*Note*: This function will automatically plot data from all conditions. If you want to change that behavior, and plot only selected conditions (or just one), you have to manually call `saliva_plot()` from the module `biopsykit.protocols.plotting` (see below).

In [ ]:
fig, ax = tsst.saliva_plot(saliva_type="cortisol", legend_loc="upper right")
fig.tight_layout()

Plot Cortisol Data – Only "Intervention" Group

In [ ]:
fig, ax = plt.subplots()
bp.protocols.plotting.saliva_plot(
    data=tsst.saliva_data["cortisol"].xs("Intervention"), 
    saliva_type="cortisol", 
    test_times=tsst.test_times, 
    test_title="TSST",
    legend_loc="upper right",
    ax=ax
)
fig.tight_layout()

Plot Cortisol and IL-6 Data

In [ ]:
fig, ax = plt.subplots()
tsst.saliva_plot(
    saliva_type=["cortisol", "il6"], 
    linestyle=["-", "--"], 
    marker=["o", "P"], 
    legend_loc="upper right", 
    ax=ax
)